<a href="https://colab.research.google.com/github/ndb796/Deep-Learning-Paper-Review-and-Practice/blob/master/code_practices/ResNet18_CIFAR10_Train.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import torch

torch.cuda.is_available()

True

In [2]:
import torch
import random
import torch.backends.cudnn as cudnn

torch.manual_seed(0)
torch.cuda.manual_seed(0)
torch.cuda.manual_seed_all(0)
cudnn.benchmark = False
cudnn.deterministic = True
random.seed(0)

#### ResNet18 모델 정의 및 인스턴스 초기화

In [3]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.backends.cudnn as cudnn
import torch.optim as optim
import os


# ResNet18을 위해 최대한 간단히 수정한 BasicBlock 클래스 정의
class BasicBlock(nn.Module):
    def __init__(self, in_planes, planes, stride=1):
        super(BasicBlock, self).__init__()

        # 3x3 필터를 사용 (너비와 높이를 줄일 때는 stride 값 조절)
        self.conv1 = nn.Conv2d(in_planes, planes, kernel_size=3, stride=stride, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        # 3x3 필터를 사용 (패딩을 1만큼 주기 때문에 너비와 높이가 동일)
        self.conv2 = nn.Conv2d(planes, planes, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn2 = nn.BatchNorm2d(planes) # 배치 정규화(batch normalization)

        self.shortcut = nn.Sequential() # identity인 경우
        if stride != 1: # stride가 1이 아니라면, Identity mapping이 아닌 경우
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_planes, planes, kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes)
            )

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.bn2(self.conv2(out))
        out += self.shortcut(x) # (핵심) skip connection
        out = F.leaky_relu(out)
        return out


# ResNet 클래스 정의
class ResNet(nn.Module):
    def __init__(self, block, num_blocks, num_classes=10):
        super(ResNet, self).__init__()
        self.in_planes = 64

        # 64개의 3x3 필터(filter)를 사용
        self.conv1 = nn.Conv2d(3, 64, kernel_size=3, stride=1, padding=1, bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.layer1 = self._make_layer(block, 64, num_blocks[0], stride=1)
        self.layer2 = self._make_layer(block, 128, num_blocks[1], stride=2)
        self.layer3 = self._make_layer(block, 256, num_blocks[2], stride=2)
        self.layer4 = self._make_layer(block, 512, num_blocks[3], stride=2)
        self.linear = nn.Linear(512, num_classes)

    def _make_layer(self, block, planes, num_blocks, stride):
        strides = [stride] + [1] * (num_blocks - 1)
        layers = []
        for stride in strides:
            layers.append(block(self.in_planes, planes, stride))
            self.in_planes = planes # 다음 레이어를 위해 채널 수 변경
        return nn.Sequential(*layers)

    def forward(self, x):
        out = F.leaky_relu(self.bn1(self.conv1(x)))
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.layer4(out)
        out = F.avg_pool2d(out, 4)
        out = out.view(out.size(0), -1)
        out = self.linear(out)
        return out


# ResNet18 함수 정의
def ResNet18():
    return ResNet(BasicBlock, [2, 2, 2, 2])

#### 데이터셋(Dataset) 다운로드 및 불러오기

In [4]:
import torchvision
import torchvision.transforms as transforms
from torch.utils.data import ConcatDataset


data_transformations = {
    "crop_flip": transforms.Compose([  # 크롭 + 뒤집기
    transforms.RandomCrop(32, padding=4),
    transforms.RandomHorizontalFlip(),
    transforms.ToTensor(),]), 

    "color_blur": transforms.Compose([  # 색깔 + 블러
     transforms.ColorJitter(brightness=(0.5, 0.9), 
                           contrast=(0.4, 0.8), 
                           saturation=(0.7, 0.9),
                           hue=(-0.2, 0.2),
                          ),
    transforms.GaussianBlur(kernel_size=(19, 19), sigma=(1.0, 2.0)),
    transforms.ToTensor()]),

    "basic": transforms.Compose([
    transforms.ToTensor()])
}

train_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, download=True, transform=data_transformations["basic"])
train_aug_dataset = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["crop_flip"])
train_aug_dataset2 = torchvision.datasets.CIFAR10(root='./data', train=True, transform=data_transformations["color_blur"])
                                                  
augmented_train_data = ConcatDataset((train_dataset,train_aug_dataset, train_aug_dataset2 ))

test_dataset = torchvision.datasets.CIFAR10(root='./data', train=False, download=True, transform=data_transformations["basic"])

train_loader = torch.utils.data.DataLoader(augmented_train_data, batch_size=128, shuffle=True, num_workers=4)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=100, shuffle=False, num_workers=4)

Files already downloaded and verified
Files already downloaded and verified


#### 환경 설정 및 학습(Training) 함수 정의

In [5]:
device = 'cuda'

net = ResNet18()
net = net.to(device)
net = torch.nn.DataParallel(net)
cudnn.benchmark = True

learning_rate = 0.1
file_name = 'resnet18_cifar10.pt'

criterion = nn.CrossEntropyLoss()
optimizer = optim.SGD(net.parameters(), lr=learning_rate, momentum=0.9, weight_decay=0.0002)


def train(epoch):
    print('\n[ Train epoch: %d ]' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(train_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        optimizer.zero_grad()

        benign_outputs = net(inputs)
        loss = criterion(benign_outputs, targets)
        loss.backward()

        optimizer.step()
        train_loss += loss.item()
        _, predicted = benign_outputs.max(1)

        total += targets.size(0)
        correct += predicted.eq(targets).sum().item()
        
        if batch_idx % 100 == 0:
            print('\nCurrent batch:', str(batch_idx))
            print('Current benign train accuracy:', str(predicted.eq(targets).sum().item() / targets.size(0)))
            print('Current benign train loss:', loss.item())

    print('\nTotal benign train accuarcy:', 100. * correct / total)
    print('Total benign train loss:', train_loss)


def test(epoch):
    print('\n[ Test epoch: %d ]' % epoch)
    net.eval()
    loss = 0
    correct = 0
    total = 0

    for batch_idx, (inputs, targets) in enumerate(test_loader):
        inputs, targets = inputs.to(device), targets.to(device)
        total += targets.size(0)

        outputs = net(inputs)
        loss += criterion(outputs, targets).item()

        _, predicted = outputs.max(1)
        correct += predicted.eq(targets).sum().item()

    print('\nTest accuarcy:', 100. * correct / total)
    print('Test average loss:', loss / total)

    state = {
        'net': net.state_dict()
    }
    if not os.path.isdir('checkpoint'):
        os.mkdir('checkpoint')
    torch.save(state, './checkpoint/' + file_name)
    print('Model Saved!')


def adjust_learning_rate(optimizer, epoch):
    lr = learning_rate
    if epoch >= 50:
        lr = 0.01
    if epoch >= 80:
        lr = 0.001
    for param_group in optimizer.param_groups:
        param_group['lr'] = lr

#### 학습(Training) 진행

* 대략 20번의 epoch 이후에도 85%가량의 test accuracy를 얻을 수 있습니다.

In [6]:
# for epoch in range(0, 200):
for epoch in range(0, 100):
    adjust_learning_rate(optimizer, epoch)
    train(epoch)
    test(epoch)


[ Train epoch: 0 ]

Current batch: 0
Current benign train accuracy: 0.078125
Current benign train loss: 2.39605975151062

Current batch: 100
Current benign train accuracy: 0.2421875
Current benign train loss: 2.07525897026062

Current batch: 200
Current benign train accuracy: 0.3203125
Current benign train loss: 1.929412841796875

Current batch: 300
Current benign train accuracy: 0.2890625
Current benign train loss: 1.947486400604248

Current batch: 400
Current benign train accuracy: 0.3828125
Current benign train loss: 1.7918590307235718

Current batch: 500
Current benign train accuracy: 0.3359375
Current benign train loss: 1.927801251411438

Current batch: 600
Current benign train accuracy: 0.390625
Current benign train loss: 1.6952723264694214

Current batch: 700
Current benign train accuracy: 0.3671875
Current benign train loss: 1.6330736875534058

Current batch: 800
Current benign train accuracy: 0.3984375
Current benign train loss: 1.6813087463378906

Current batch: 900
Current 


Current batch: 800
Current benign train accuracy: 0.8203125
Current benign train loss: 0.4644944369792938

Current batch: 900
Current benign train accuracy: 0.859375
Current benign train loss: 0.3271680176258087

Current batch: 1000
Current benign train accuracy: 0.890625
Current benign train loss: 0.3514084815979004

Current batch: 1100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.37813565135002136

Total benign train accuarcy: 84.46333333333334
Total benign train loss: 521.0203751027584

[ Test epoch: 5 ]

Test accuarcy: 80.71
Test average loss: 0.005855456122756004
Model Saved!

[ Train epoch: 6 ]

Current batch: 0
Current benign train accuracy: 0.875
Current benign train loss: 0.3491497337818146

Current batch: 100
Current benign train accuracy: 0.8828125
Current benign train loss: 0.3283672034740448

Current batch: 200
Current benign train accuracy: 0.8671875
Current benign train loss: 0.45819273591041565

Current batch: 300
Current benign train accuracy:


Current batch: 200
Current benign train accuracy: 0.8828125
Current benign train loss: 0.2840496003627777

Current batch: 300
Current benign train accuracy: 0.8828125
Current benign train loss: 0.38442718982696533

Current batch: 400
Current benign train accuracy: 0.890625
Current benign train loss: 0.34207648038864136

Current batch: 500
Current benign train accuracy: 0.8671875
Current benign train loss: 0.35407787561416626

Current batch: 600
Current benign train accuracy: 0.859375
Current benign train loss: 0.37818479537963867

Current batch: 700
Current benign train accuracy: 0.890625
Current benign train loss: 0.2828763425350189

Current batch: 800
Current benign train accuracy: 0.890625
Current benign train loss: 0.2932911217212677

Current batch: 900
Current benign train accuracy: 0.8984375
Current benign train loss: 0.34458059072494507

Current batch: 1000
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2759883999824524

Current batch: 1100
Current benign


Current batch: 1000
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2571302056312561

Current batch: 1100
Current benign train accuracy: 0.9375
Current benign train loss: 0.20389977097511292

Total benign train accuarcy: 90.88133333333333
Total benign train loss: 309.633043423295

[ Test epoch: 16 ]

Test accuarcy: 84.19
Test average loss: 0.0053861309289932255
Model Saved!

[ Train epoch: 17 ]

Current batch: 0
Current benign train accuracy: 0.9296875
Current benign train loss: 0.19520296156406403

Current batch: 100
Current benign train accuracy: 0.8828125
Current benign train loss: 0.37981316447257996

Current batch: 200
Current benign train accuracy: 0.96875
Current benign train loss: 0.10793150216341019

Current batch: 300
Current benign train accuracy: 0.921875
Current benign train loss: 0.20076265931129456

Current batch: 400
Current benign train accuracy: 0.9140625
Current benign train loss: 0.23552384972572327

Current batch: 500
Current benign train acc


Current batch: 400
Current benign train accuracy: 0.90625
Current benign train loss: 0.2664947509765625

Current batch: 500
Current benign train accuracy: 0.859375
Current benign train loss: 0.34808698296546936

Current batch: 600
Current benign train accuracy: 0.9609375
Current benign train loss: 0.1899157017469406

Current batch: 700
Current benign train accuracy: 0.9453125
Current benign train loss: 0.16064107418060303

Current batch: 800
Current benign train accuracy: 0.890625
Current benign train loss: 0.3693056106567383

Current batch: 900
Current benign train accuracy: 0.9296875
Current benign train loss: 0.24795761704444885

Current batch: 1000
Current benign train accuracy: 0.8984375
Current benign train loss: 0.2950933277606964

Current batch: 1100
Current benign train accuracy: 0.8984375
Current benign train loss: 0.29062333703041077

Total benign train accuarcy: 91.584
Total benign train loss: 289.49459214136004

[ Test epoch: 22 ]

Test accuarcy: 84.71
Test average loss: 

...


Current batch: 600
Current benign train accuracy: 0.9921875
Current benign train loss: 0.012492687441408634

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.005449328105896711

Current batch: 800
Current benign train accuracy: 1.0
Current benign train loss: 0.0062340605072677135

Current batch: 900
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02078869566321373

Current batch: 1000
Current benign train accuracy: 0.9921875
Current benign train loss: 0.03372716158628464

Current batch: 1100
Current benign train accuracy: 0.984375
Current benign train loss: 0.028942300006747246

Total benign train accuarcy: 99.58533333333334
Total benign train loss: 16.532770311518107

[ Test epoch: 72 ]

Test accuarcy: 92.94
Test average loss: 0.0027368134424090385
Model Saved!

[ Train epoch: 73 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0094240577891469

Current batch: 100
Current benign train accuracy:


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.0035328397061675787

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.00788910686969757

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.005328819155693054

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.0060794358141720295

Current batch: 500
Current benign train accuracy: 0.9921875
Current benign train loss: 0.02525784634053707

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.004972557537257671

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.016837559640407562

Current batch: 800
Current benign train accuracy: 0.9765625
Current benign train loss: 0.04238780587911606

Current batch: 900
Current benign train accuracy: 0.9765625
Current benign train loss: 0.05258022993803024

Current batch: 1000
Current benign train accuracy: 0.992


Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.0028000776655972004

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.001084144227206707

Total benign train accuarcy: 99.85
Total benign train loss: 7.459131405630615

[ Test epoch: 83 ]

Test accuarcy: 93.4
Test average loss: 0.0025429425194859506
Model Saved!

[ Train epoch: 84 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.008464319631457329

Current batch: 100
Current benign train accuracy: 0.9921875
Current benign train loss: 0.006767515558749437

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0028671505860984325

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.0012169622350484133

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.00033656248706392944

Current batch: 500
Current benign train accuracy: 1.0
Current benign tra


Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0008326450479216874

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.0035935095511376858

Current batch: 800
Current benign train accuracy: 1.0
Current benign train loss: 0.0031561797950416803

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.002746936399489641

Current batch: 1000
Current benign train accuracy: 1.0
Current benign train loss: 0.0015108016086742282

Current batch: 1100
Current benign train accuracy: 1.0
Current benign train loss: 0.000660365039948374

Total benign train accuarcy: 99.90933333333334
Total benign train loss: 5.2447643872583285

[ Test epoch: 89 ]

Test accuarcy: 93.54
Test average loss: 0.0025016103111207486
Model Saved!

[ Train epoch: 90 ]

Current batch: 0
Current benign train accuracy: 1.0
Current benign train loss: 0.0022048763930797577

Current batch: 100
Current benign train accuracy: 1.0
Current be


Current batch: 100
Current benign train accuracy: 1.0
Current benign train loss: 0.00181449088267982

Current batch: 200
Current benign train accuracy: 1.0
Current benign train loss: 0.0009719214285723865

Current batch: 300
Current benign train accuracy: 1.0
Current benign train loss: 0.008777364157140255

Current batch: 400
Current benign train accuracy: 1.0
Current benign train loss: 0.000836374529171735

Current batch: 500
Current benign train accuracy: 1.0
Current benign train loss: 0.0011358835035935044

Current batch: 600
Current benign train accuracy: 1.0
Current benign train loss: 0.0008843070827424526

Current batch: 700
Current benign train accuracy: 1.0
Current benign train loss: 0.003290606662631035

Current batch: 800
Current benign train accuracy: 1.0
Current benign train loss: 0.0016765902983024716

Current batch: 900
Current benign train accuracy: 1.0
Current benign train loss: 0.0019366564229130745

Current batch: 1000
Current benign train accuracy: 1.0
Current benig